In [1]:
import networkx as nx
import osmnx as ox
from shapely.geometry import LineString, mapping
import geopandas as gpd
from ipyleaflet import *
import pandas as pd
import warnings
warnings.simplefilter('ignore')
import panel as pn
from panel.interact import interact, interactive, fixed, interact_manual
from panel import widgets
pn.extension()
from ipywidgets import HTML
from datetime import datetime

In [2]:
time_now = datetime.now().strftime("%H")
print("The current date and time is", time_now)

The current date and time is 11


In [3]:
if (int(time_now) >= 6 and int(time_now) <= 10) or (int(time_now) >= 17 and int(time_now) <= 21):
    horas_punta = ['Avenida Panamericana Norte', 'Autopista Chillón - Trapiche', 'Auxiliar Chillón - Trapiche', 'Avenida Universitaria', 'Avenida Gerardo Unger', 'Avenida Túpac Amaru', 'Jirón Puno']
else: horas_punta = []

In [4]:
import matplotlib.pyplot as plt
%matplotlib inline
graph = ox.graph.graph_from_xml(filepath='D:/Documentos/Complejidad Algorítmica/HITO2/graph.osm')

In [5]:
#fig, ax = ox.plot_graph(graph)

In [6]:
nodes, edges = ox.graph_to_gdfs(graph)

In [7]:
excluded_graph = []
excluded_graph.append(ox.graph_from_gdfs(nodes, edges))

In [8]:
widgetExcluded = pn.widgets.CheckButtonGroup(name='Excluir Rutas Congestionadas', value=[], options=['Avenida Panamericana Norte', 'Autopista Chillón - Trapiche', 'Auxiliar Chillón - Trapiche', 'Avenida Universitaria', 'Avenida El Retablo', 'Avenida Los Angeles', 'Avenida Gerardo Unger', 'Avenida Alfredo Mendiola', 'Avenida Túpac Amaru', 'Avenida Micaela Bastidas', 'Avenida Jose Pardo', 'Avenida 22 de Agosto', 'Jirón Puno', 'Avenida Arequipa', 'Avenida Jorge Chavez'])

In [9]:
#widgetExcluded.value

In [10]:
buttonExcluded = pn.widgets.Button(name='Guardar Excluidos', button_type='primary')

In [11]:
def ExcludeEdges():
    nodos,aristas = ox.graph_to_gdfs(graph)
    for i in excluded_array:
        aristas = aristas[aristas['name']!=i]
    excluded_graph.append(ox.graph_from_gdfs(nodos, aristas))

In [12]:
excluded_array=[]
def Excluir(event):
    excluded_array.clear()
    excluded_graph.clear()
    for i in widgetExcluded.value:
        excluded_array.append(str(i))
    ExcludeEdges()
buttonExcluded.on_click(Excluir)

Watcher(inst=Button(button_type='primary', name='Guardar Excluidos'), cls=<class 'panel.widgets.button.Button'>, fn=<function Excluir at 0x000001E9429F8040>, mode='args', onlychanged=False, parameter_names=('clicks',), what='value', queued=False, precedence=0)

In [13]:
center = (-11.92063, -77.05529)
m = Map(center=center, basemap=basemaps.OpenStreetMap.HOT, zoom=15)
m.add_control(FullScreenControl())

In [14]:
#style for the destination marker
to_marker_style = AwesomeIcon(
    name='map-pin',
    icon_color='red',
    spin=True
)
marker_style = Icon (
    icon_url = 'https://cdn.pixabay.com/photo/2014/04/03/10/03/google-309741_960_720.png'
)
from_marker = Marker(location=center)
mid_marker = Marker(location = center, icon=to_marker_style)
to_marker = Marker(location=center, icon=marker_style)

In [15]:
def set_nearest_node(marker):
 marker.nearest_node = ox.get_nearest_node(graph, marker.location)
 return

In [16]:
from networkx.generators.classic import null_graph
path_layer_from_list = []
path_layer_to_list = []
distancia = []
output = ['...']
def handle_change_location(event, marker, algorithm, distancia, desde):
    event_owner = event['owner']
    event_owner.nearest_node = ox.get_nearest_node(graph, event_owner.location)
    marker.nearest_node = ox.get_nearest_node(graph, marker.location)

    shortest_path = null_graph
    length = 0

    if algorithm == 'dijkstra':
      shortest_path = nx.dijkstra_path(excluded_graph[0], event_owner.nearest_node, marker.nearest_node, 
                                     weight='length')
      length = nx.dijkstra_path_length(excluded_graph[0], event_owner.nearest_node, marker.nearest_node, 
                                     weight='length')
      colour = 'black'
    elif algorithm == 'astar':
      shortest_path = nx.astar_path(excluded_graph[0], event_owner.nearest_node, marker.nearest_node, 
                                     weight='length')
      length = nx.astar_path_length(excluded_graph[0], event_owner.nearest_node, marker.nearest_node, 
                                     weight='length')
      colour = 'blue'
      
    
    if marker == from_marker and len(path_layer_from_list) == 1:
      m.remove_layer(path_layer_from_list[0])
      path_layer_from_list.pop()

    elif marker == to_marker and len(path_layer_to_list) == 1:
      m.remove_layer(path_layer_to_list[0])
      path_layer_to_list.pop()
      
    elif marker == mid_marker:
      if len(path_layer_from_list) == 1:
        m.remove_layer(path_layer_from_list[0])
        path_layer_from_list.pop()
      if len(path_layer_to_list) == 1:
        m.remove_layer(path_layer_to_list[0])
        path_layer_to_list.pop()
    
    shortest_path_points = nodes.loc[shortest_path]
    path = gpd.GeoDataFrame([LineString(shortest_path_points.geometry.values)], columns=['geometry'])
    path_layer = GeoData(geo_dataframe=path, style={'color':colour, 'weight':2})
    if marker == from_marker:
      m.add_layer(path_layer)
      path_layer_from_list.append(path_layer)
    elif marker == to_marker:
      m.add_layer(path_layer)
      path_layer_to_list.append(path_layer)

    if (marker == from_marker or marker == to_marker) and len(distancia) < 2:
      distancia.append(length)
    else: distancia.clear()

    if len(distancia) == 2:
        output_popup.value=str(algorithm +':' + str(round(distancia[0] + distancia[1], 2)) + ' metros, ' + str(round((distancia[0] + distancia[1]) / 666.666666667, 2)) + ' minutos')

In [17]:
m.add_layer(from_marker)
m.add_layer(mid_marker)
m.add_layer(to_marker)
set_nearest_node(from_marker)
set_nearest_node(mid_marker)
set_nearest_node(to_marker)
output_popup = HTML()
output_popup.value = ''
from_marker.popup=output_popup
mid_marker.popup=output_popup
to_marker.popup=output_popup

In [18]:
widget = pn.widgets.RadioButtonGroup(name='Algorithm', options=['Dijkstra', 'A*'], button_type='success')

In [19]:
def change_algorithm(algorithm):
    if algorithm == 'Dijkstra': 
        algorithm_setter = 'dijkstra'
        from_marker.observe(lambda event: handle_change_location(event, mid_marker, 'dijkstra', distancia, 1), 'location')
        mid_marker.observe(lambda event: handle_change_location(event, from_marker, 'dijkstra', distancia, 2), 'location')
        mid_marker.observe(lambda event: handle_change_location(event, to_marker, 'dijkstra', distancia, 3), 'location')
        to_marker.observe(lambda event: handle_change_location(event, mid_marker, 'dijkstra', distancia, 4), 'location')
    elif algorithm == 'A*': 
        algorithm_setter = 'astar' 
        from_marker.observe(lambda event: handle_change_location(event, mid_marker, 'astar', distancia, 1), 'location')
        mid_marker.observe(lambda event: handle_change_location(event, from_marker, 'astar', distancia, 2), 'location')
        mid_marker.observe(lambda event: handle_change_location(event, to_marker, 'astar', distancia, 3), 'location')
        to_marker.observe(lambda event: handle_change_location(event, mid_marker, 'astar', distancia, 4), 'location')
    display(m)
    
pn.Column('Algoritmos:', interact(change_algorithm, algorithm=widget), 'Excluir Rutas', 'Las siguientes rutas están congestionadas:', horas_punta, widgetExcluded, buttonExcluded)

Map(center=[-11.92063, -77.05529], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

Column
    [0] Markdown(str)
    [1] Column
        [0] Column
            [0] RadioButtonGroup(button_type='success', name='Algorithm', options=['Dijkstra', 'A*'], value='Dijkstra')
        [1] Row
            [0] Str(None, name='interactive00104')
    [2] Markdown(str)
    [3] Markdown(str)
    [4] JSON(list)
    [5] CheckButtonGroup(name='Excluir Rutas C..., options=['Avenida Panamericana Nor...])
    [6] Button(button_type='primary', name='Guardar Excluidos')